# Importación


In [33]:
import pandas as pd
import numpy as np 
import plotly as ply 
import openai as openai
import keyfile as kf

In [2]:
df_RatedFlights = pd.read_csv('rated_flights.csv')



Los datos presentados corresponden a algunos de los vuelos sucitados entre los años 2002-2024, en el cual se destaca como variable categórica el overall rating el cual ponderiza el rating que las personas le dieron a su experiencia en el check-in, waiting and boarding, in flight y arrival 

Los únicos datos con un componente "subjetivo" son el overall rating y los remarks dentro del reportLink, ya que reflejan la percepción individual de cada experiencia.

# Limpieza

Los datos ingresados manualmente se validan antes de incluirse en el reporte, verificando aerolíneas y ciudades según sus códigos. Además, el formato se asegura automáticamente mediante un formulario con validación en tiempo real

In [3]:
nans= df_RatedFlights.isnull().sum()
print(nans)

#no missing values in the dataset

Date             0
Airline          0
Departure        0
Arrival          0
OverallRating    0
ReportLink       0
dtype: int64


In [4]:
duplicates= df_RatedFlights.duplicated()

print(duplicates.sum())

#no duplicates in the dataset

0


# EDA

Existencia de vuelos que partan de una ciudad y regresen a la misma

In [5]:
sameDepArrFlights= df_RatedFlights['Departure'] == df_RatedFlights['Arrival']
print("vuelos de origen y destino igual:",sameDepArrFlights.sum())

vuelos de origen y destino igual: 9


In [6]:
df_RatedFlights[sameDepArrFlights]

,Date,Airline,Departure,Arrival,OverallRating,ReportLink
688,25/05/2022,Sundair,DRSDresden Klotzsche Airport,DRSDresden Klotzsche Airport,10,../flightlog/report_545086_Sundair_1960.htm
701,18/04/2022,Sundair,LEJLeipzig,LEJLeipzig,8,../flightlog/report_542960_Sundair_8796.htm
756,05/02/2022,Air New Zealand,AKLAuckland Auckland International Airport,AKLAuckland Auckland International Airport,10,../flightlog/report_537770_Air New Zealand_601...
760,19/01/2022,Air New Zealand,VCVVictorville George AFB - USA California,VCVVictorville George AFB - USA California,10,../flightlog/report_536960_Air New Zealand_600...
821,16/10/2021,Danish Air Transport,CPHCopenhagen Kastrup,CPHCopenhagen Kastrup,10,../flightlog/report_530715_Danish Air Transpor...
970,07/11/2020,Eurowings,TXLBerlin Tegel,TXLBerlin Tegel,8,../flightlog/report_508926_Eurowings_5237.htm
1735,04/03/2017,Delta Air Lines,ATLAtlanta Hartsfield Jackson International - ...,ATLAtlanta Hartsfield Jackson International - ...,9,../flightlog/report_389510_Delta Air Lines_31.htm
2264,11/11/2014,KLM,AMSAmsterdam Schiphol,AMSAmsterdam Schiphol,10,../flightlog/report_339228_KLM_9895.htm
3031,26/11/2010,KLM,AMSAmsterdam Schiphol,AMSAmsterdam Schiphol,8,../flightlog/report_391038_KLM_803.htm


Se descartan, pues, carecen de confiabilidad

In [7]:
df_RatedFlights.drop(df_RatedFlights[sameDepArrFlights].index, inplace=True)

Análisis de las aerolineas en el dataframe

In [8]:
uniqueArilines = df_RatedFlights["Airline"].unique()

print(f"hay {uniqueArilines.size} aerolineas registradas")

hay 313 aerolineas registradas


In [36]:
#convertimos en csv el uniqueArilines
# uniqueArilines.sort()
# df_Airlines = pd.DataFrame(uniqueArilines, columns=['Airline'])
# df_Airlines.to_csv('airlines.csv', index=False)


Obtenemos los paises de los que son cada una de las aerolineas puesto que es una variable que puede influir en el OverallRating

In [39]:
df_Airlines = pd.read_csv('airlines.csv')

df_RatedFlights["Country_Airline"] = df_RatedFlights["Airline"].map(df_Airlines.set_index('Airline')['Country'])

df_RatedFlights

,Date,Airline,Departure,Arrival,OverallRating,ReportLink,Country_Airline
0,19/01/2025,LATAM Brasil,NATNatal Greater Natal International Airport,GRUSao Paulo Guarulhos Int'l,7,../flightlog/report_625375_LATAM Brasil_3443.htm,Brasil
1,13/01/2025,KLM,BCNBarcelona El Prat,AMSAmsterdam Schiphol,8,../flightlog/report_624841_KLM_1518.htm,Países Bajos
2,11/01/2025,Azul Linhas Aereas,SSASalvador Deputado Luís Eduardo Magalhães In...,RECRecife Guararapes - Gilberto Freyre Int'l,10,../flightlog/report_625374_Azul Linhas Aereas_...,Brasil
3,09/01/2025,Garuda,SINSingapore Changi,CGKJakarta Soekarno - Hatta,9,../flightlog/report_624575_Garuda_839.htm,Indonesia
4,08/01/2025,Azul Linhas Aereas,GRUSao Paulo Guarulhos Int'l,IOSIlheus,10,../flightlog/report_625373_Azul Linhas Aereas_...,Brasil
...,...,...,...,...,...,...,...
3095,25/05/2003,DutchBird,LPALas Palmas De Gran Canaria Gando,AMSAmsterdam Schiphol,7,../flightlog/report_221289_DutchBird_902.htm,Países Bajos
3096,12/05/2003,DutchBird,AMSAmsterdam Schiphol,LPALas Palmas De Gran Canaria Gando,6,../flightlog/report_221286_DutchBird_901.htm,Países Bajos
3097,12/05/2003,DutchBird,LPALas Palmas De Gran Canaria Gando,FORFortaleza Pinto Martins,7,../flightlog/report_221287_DutchBird_901.htm,Países Bajos
3098,07/02/2003,IrAero,CGKJakarta Soekarno - Hatta,SUBSurabaya Juanda,7,../flightlog/report_339682_IrAero_200.htm,Rusia


In [40]:
df_plane_crashes = pd.read_csv('plane_crashes.csv')


ParserError: Error tokenizing data. C error: Expected 5 fields in line 102, saw 6


In [34]:

key = kf.getKey()
openai.api_key = key

prompt = "necesito "
